In [1]:
%load_ext Cython

In [ ]:
# import numpy as np
# results = np.zeros((50,50))
# np.savetxt("same_start_fast_test.txt", results)

In [ ]:
%%cython

import itertools as it
cimport cython
from localFuncs import *

@cython.cdivision(True)
cdef list get_orderToAdd(int q):
    cdef list orderToAdd = []
    cdef int a = q - 1
    cdef int b = 0
    while (a > b):
        orderToAdd += [a, b]
        b += 1
        a = q - b
    return orderToAdd[1:]

@cython.cdivision(True)
cdef list all_checks(int q):
    return list(it.combinations_with_replacement(range(q), r = 2))

@cython.cdivision(True)
cdef int find_first_el(set arithSet, int mod):
    cdef int a = 0
    for a in arithSet:
        if ((a - 2) % mod) not in arithSet:
            return a
    raise Exception(f"arithSet was too big or was a complete cyclic progression in Z_{mod}")

@cython.cdivision(True)
cdef list mod_sort(set nums, int mod):
    cdef list toSort = [num + mod if num < 0 else num for num in nums]
    cdef list sort = sorted(toSort)
    cdef int breakPoint = len(sort)
    cdef int i = 0
    for i in range(len(sort) - 1):
        if sort[i+1] > sort[i] + 2:
            breakPoint = i + 1
    cdef list out = [num - mod for num in sort[breakPoint:len(sort)]] + sort[:breakPoint]
    return sorted(out)
    

cdef fine(list A, tuple rowTup, q, w):
    cdef set sumSet
    cdef int row1 = rowTup[0]
    cdef int row2 = rowTup[1]
    if row1 == row2:
        sumSet = {(a + b) % (q * w) for a in A[row1] for b in A[row2] if a != b}
    else:
        sumSet = {(a + b) % (q * w) for a in A[row1] for b in A[row2]}
    return len(sumSet.intersection(A[(row1 + row2) % q])) == 0


# cdef find_same_start_arith(q, w, sizeTup, verbose=False):
#     cdef list ordered_range_q = get_orderToAdd(q)
#     cdef list left_to_be_checked, A
#     cdef int row1, row2, shift, r
#     cdef bint failed

#     for shift in range(q * w):
#         failed = False
#         left_to_be_checked = all_checks(q)
#         A = [{}] * q
#         for r in ordered_range_q:
#             A[r] = {(shift + 2 * i) % (q * w) for i in range(sizeTup[r])}
#             for rowTup in filter(lambda rowTup: len(A[rowTup[0]]) != 0 and len(A[rowTup[1]]) != 0, left_to_be_checked):
#                 left_to_be_checked.remove(rowTup)
#                 if not fine(A, rowTup, q, w):
#                     failed = True
#                     break
#             if failed:
#                 break
#         if not failed:
#             if verbose:
#                 display(Markdown("$A_{0}=\{0\} \\times \{" + ", ".join(str(el) for el in mod_sort(A[0], q*w)) + "\}$\n"))
#                 display(Markdown("$A_{1}=\{1\} \\times \{" + ", ".join(str(el) for el in mod_sort(A[1], q*w)) + "\}$\n"))
#                 display(Markdown("$\\vdots$"))
#                 display(Markdown("$A_{" + f"{q-1}" + "}=\{" + f"{q-1}" + "\} \\times \{" + ", ".join(str(el) for el in mod_sort(A[q-1], q*w)) + "\}$\n"))
#                 print("\n\nA = { " + ", ".join(", ".join(f"({i},{j})" for j in A[i]) for i in range(q)) + " }")
#             return True
#     return False



cdef find_and_return_same_start_arith(q, w, sizeTup, verbose=False):
    cdef list ordered_range_q = get_orderToAdd(q)
    cdef list left_to_be_checked, A
    cdef int row1, row2, shift, r
    cdef bint failed

    for shift in range(q * w):
        failed = False
        left_to_be_checked = all_checks(q)
        A = [{}] * q
        for r in ordered_range_q:
            A[r] = {(shift + 2 * i) % (q * w) for i in range(sizeTup[r])}
            for rowTup in filter(lambda rowTup: len(A[rowTup[0]]) != 0 and len(A[rowTup[1]]) != 0, left_to_be_checked):
                left_to_be_checked.remove(rowTup)
                if not fine(A, rowTup, q, w):
                    failed = True
                    break
            if failed:
                break
        if not failed:
            if verbose:
                display(Markdown("$A_{0}=\{0\} \\times \{" + ", ".join(str(el) for el in mod_sort(A[0], q*w)) + "\}$\n"))
                display(Markdown("$A_{1}=\{1\} \\times \{" + ", ".join(str(el) for el in mod_sort(A[1], q*w)) + "\}$\n"))
                display(Markdown("$\\vdots$"))
                display(Markdown("$A_{" + f"{q-1}" + "}=\{" + f"{q-1}" + "\} \\times \{" + ", ".join(str(el) for el in mod_sort(A[q-1], q*w)) + "\}$\n"))
                print("\n\nA = { " + ", ".join(", ".join(f"({i},{j})" for j in A[i]) for i in range(q)) + " }")
            return shift
    return None


cdef has_pd23(int n):
    cdef int d
    for d in primeD(n):
        if d % 3 == 2:
            return True
    return False



cdef int q_min = 3
cdef int q_max = 99 # inclusive
cdef int w_min = 1
cdef int w_max = 99 # inclusive

cdef results = np.loadtxt("same_start_fast_test_shift.txt")

for r in range(len(results)):
    for c in range(len(results[0])):
        if results[r][c] == 3.1:
            results[r][c] = 0.1
    

np.savetxt("same_start_fast_test_shift.txt", results)
     
    
    
cdef int q, w


    
# for q in range(q_min,q_max+2,2):
#     for w in range(w_min,w_max+2,2):
#         if not has_pd23(q * w):
#         if results[(q-1)//2][(w-1)//2] == 0.1:
#             results[int(q-1)//2][(w-1)//2] = 3.1
#             np.savetxt("same_start_fast_test_shift.txt", results)
#             mu = mu21((q, q*w))
#             t = (mu//q + 1,) + (mu//q,)*(q-1)
#             if find_same_start_arith(q, w, t):
#                 results[(q-1)//2][(w-1)//2] = 1.1 # found
#             else:
#                 results[(q-1)//2][(w-1)//2] = 2.1 # not found
#             np.savetxt("same_start_fast_test_shift.txt", results)
# print("Done.                   ")




# for q in range(q_min,q_max+2,2):
#     for w in range(w_min,w_max+2,2):
#         if (not has_pd23(q * w)) and ((q * w) % 6 == 1) and results[int(q-1)//2][(w-1)//2] > 0:
#             results[int(q-1)//2][(w-1)//2] = 3.1
#             np.savetxt("same_start_fast_test_shift.txt", results)
#             mu = mu21((q, q*w))
#             t = (mu//q + 1,) + (mu//q,)*(q-1)
#             results[int(q-1)//2][(w-1)//2] = find_and_return_same_start_arith(q, w, t) -  q * w
#             np.savetxt("same_start_fast_test_shift.txt", results)
# print("Done.                   ")





